In [1]:
'''
TODO:
1. Leverage `sentence-transformers/all-MiniLM-L6-v2` for embedding
2. Convert CSV Dataset to string
3. Compute Sentence Embedding
4. Build KNN index using FAISS
5. Process an input sentence and find k nearest sentences
6. Feed retrieves sentences to LLM
'''

'\nTODO:\n1. Leverage `sentence-transformers/all-MiniLM-L6-v2` for embedding\n2. Convert CSV Dataset to string\n3. Compute Sentence Embedding\n4. Build KNN index using FAISS\n5. Process an input sentence and find k nearest sentences\n6. Feed retrieves sentences to LLM\n'

***Sentence Transformer***

In [1]:
from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# The sentences to encode
sentences = [
    "The weather is lovely today.",
    "It's so sunny outside!",
    "He drove to the stadium.",
]

# 2. Calculate embeddings by calling model.encode()
embeddings = model.encode(sentences)
print(embeddings.shape)
# [3, 384]

# 3. Calculate the embedding similarities
similarities = model.similarity(embeddings, embeddings)
print(similarities)
# tensor([[1.0000, 0.6660, 0.1046],
#         [0.6660, 1.0000, 0.1411],
#         [0.1046, 0.1411, 1.0000]])


/media/iscrea/01DAF8607170AED0/LinuxMint/Code/AI/RAG-Ecoms/ragecoms-venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(3, 384)
tensor([[1.0000, 0.6660, 0.1046],
        [0.6660, 1.0000, 0.1411],
        [0.1046, 0.1411, 1.0000]])


In [3]:
import pandas as pd

***Read Data***

In [10]:
dir = '../data/Beauty/all.csv'
df = pd.read_csv(dir)

In [11]:
df.head()

,user_id,item_id,rating,timestamp,summary,neg_item_id,position,history_item_id,history_rating,history_summary,history,user_profile,target_item_attributes,candidate_item_id,candidate_item_attributes
0,7188,104,5.0,1023840000,Gentle Hand Soap,"[177, 10080, 9114, 8743, 9727, 11504, 381, 809...",0,[],[],[],NaN,unknown,"Brand: Jason Natural, Price: 6.49, Categories:...","[10080, 9114, 381, 177, 8743, 9727, 8095, 104,...","10080: Brand: unknown, Price: 16.95, Categorie..."
1,7188,101,5.0,1024185600,Only shampoo my husband will use lately,"[5302, 9595, 8472, 1693, 1098, 8870, 11050, 58...",1,[104],[5.0],['Gentle Hand Soap'],"Brand: Jason Natural, Price: 6.49, Categories:...",unknown,"Brand: Jason Natural, Price: 5.56, Categories:...","[101, 11050, 9595, 5302, 1098, 1693, 8870, 755...","101: Brand: Jason Natural, Price: 5.56, Catego..."
2,7188,102,5.0,1024185600,Good detangler,"[5623, 583, 10773, 9278, 3451, 1697, 8044, 725...",2,"[104, 101]","[5.0, 5.0]","['Gentle Hand Soap', 'Only shampoo my husband ...","Brand: Jason Natural, Price: 6.49, Categories:...",unknown,"Brand: Jason Natural, Price: 10.46, Categories...","[10773, 3451, 583, 9278, 102, 5623, 9560, 8044...","10773: Brand: Nexxus, Price: 12.97, Categories..."
3,21815,152,5.0,1036627200,I am SO EXCITED!,"[1608, 11133, 9196, 9578, 5142, 352, 648, 1413...",0,[],[],[],NaN,unknown,"Brand: Vidal Sassoon, Price: 38.95, Categories...","[352, 152, 9196, 1608, 648, 9436, 9578, 11133,...","352: Brand: Soft &#39;N Style, Price: 3.65, Ca..."
4,7188,23,5.0,1052611200,Use as an extra ingredient in your own mixture,"[8234, 2696, 10343, 920, 6619, 3299, 11507, 41...",3,"[104, 101, 102]","[5.0, 5.0, 5.0]","['Gentle Hand Soap', 'Only shampoo my husband ...","Brand: Jason Natural, Price: 6.49, Categories:...",unknown,"Brand: Alberto VO5, Price: 26.59, Categories: ...","[2696, 4101, 11507, 6619, 10343, 4967, 23, 329...","2696: Brand: L&#39;Oreal Paris, Price: 0.8, Ca..."


In [ ]:
information = df['user_id', 'item_id', 'summary', 'history', 'target_item_attributes']